# Homework 2 - Higher Order Functions (15 pts)

Please turn in your note book naming as *BDM\_HW2\_HOF_LastName.ipynb*.


## Task 1 (2 points)

Please complete the **lambda f1** definition below by filling in the _________ part. **f1** is expected to take a single string argument and returns whether the string can be converted to a natural number or zero. If it's possible, returns **'Number'**, otherwise returns **'Not a number'**. For example *'0123'* is a valid number 'Number', whereas *'0xff'* is not.

In [1]:
# String has an inbuilt checker for whether a string is numeric: str.isnumeric()
f1 = lambda x: 'Number' if x.isnumeric() else 'Not a Number'
print(f1('1A'))
print(f1('12'))
print(f1('b1'))
print(f1('0xff')) # Added case to make sure

Not a Number
Number
Not a Number
Not a Number



## Task 2 (2 points)

Please complete the **lambda f2** definition below by filling in the _________ part. **f2** takes a single iterable (e.g. a list or a string), and returns the number of unique elements. Please see the sample output below.

In [2]:
# We can make a set and take the len of that set
f2 = lambda x: len(set(x))

print(f2([1,2,3,4,1,5,2]))
print(f2('hello world'))

5
8



## Task 3 (2 points)

Please complete the **lambda f3** definition below by filling in the _________ part. **f3** takes two strings *x* and *y*, and return all the words in *x* that do not appear in *y*. Please note that the word comparison iscase insensitive. Samples area also provided below.

In [3]:
getwords = lambda words : [word.lower() for word in words.split(' ')]
filterempty = lambda words : list(filter(lambda word: word != '', words))

f3 = lambda x, y: filterempty( [xword if xword not in getwords(y) else '' for xword in getwords(x)] )
print(f3('big data management and analysis', 'big data computing'))
print(f3('this is a phrase', 'this is another phrase'))

['management', 'and', 'analysis']
['a']



## Task 4 (9 points)

Please complete Homework 1 using only high order functions. 
You will need access to the **sale.csv** data set from Homework 1, or you can download them from Blackboard (under *Data Sets*).

Your solution must take a single *sales* variable as the input and transform through a series of **map()**, **filter()**, **sorted()**, and/or **reduce()** transformation. You can write helper functions to be used with HOF but you **cannot make access to global variables**.

In [4]:
import csv
from functools import reduce

# Read data into 'sales', which should contains a 'list' of 'dict'
sales = list(map(dict,csv.DictReader(open('sale.csv', 'r'))))
sales

[{'Customer ID': '1431105976',
  'Transaction ID': 'T20066',
  'Date': '2014/00/01',
  'Product ID': 'P08719',
  'Item Cost': '14.99'},
 {'Customer ID': '1431105976',
  'Transaction ID': 'T47961',
  'Date': '2014/09/03',
  'Product ID': 'P56063',
  'Item Cost': '7.99'},
 {'Customer ID': '1431105976',
  'Transaction ID': 'T24352',
  'Date': '2015/00/24',
  'Product ID': 'P41420',
  'Item Cost': '19.99'},
 {'Customer ID': '1431105976',
  'Transaction ID': 'T06252',
  'Date': '2016/08/10',
  'Product ID': 'P58928',
  'Item Cost': '16.99'},
 {'Customer ID': '1431105976',
  'Transaction ID': 'T47991',
  'Date': '2016/09/00',
  'Product ID': 'P04501',
  'Item Cost': '14.99'},
 {'Customer ID': '1431105976',
  'Transaction ID': 'T19854',
  'Date': '2016/03/14',
  'Product ID': 'P04501',
  'Item Cost': '15.99'},
 {'Customer ID': '1431105976',
  'Transaction ID': 'T00079',
  'Date': '2014/01/17',
  'Product ID': 'P08719',
  'Item Cost': '12.99'},
 {'Customer ID': '1431105976',
  'Transaction ID'

In [5]:
# At the start, each customer ID can have multiple Product IDs. This gets the customer
# to only a single product ID, and aggregates their item cost.
# If on Hadoop, we also can't use the fact that data is sorted by Customer ID, so we have to keep
# a customer set. But since this will only run on one single computer, we will still use that fact
def insertProduct(finalTable, cur):
    prodId = cur['Product ID']
    totalCost = float(cur['Item Cost'])
    custId = cur['Customer ID']
    if prodId in finalTable:
        # We are using the fact that customerIds are gaurenteed to be clustered
        # Thus we can store only the last customer, and not an entire customer set
        if prodId in finalTable:
            # We are using the fact that customerIds are gaurenteed to be clustered
            # Thus we can store only the last customer, and not an entire customer set
            if custId != str(finalTable[prodId]['lastCust']):
                finalTable[prodId]['custCount'] += 1
                finalTable[prodId]['lastCust'] = custId
            finalTable[prodId]['totalRev'] += totalCost
    else:
        finalTable[prodId] = { 'custCount': 1, 'totalRev': totalCost, 'lastCust': custId }
    return finalTable

# NOTE: If the data in insertProduct was processed split and in chunks, then we would need another step here
# to combine that data.

# Now we convert this list to the format we want
def convertInput(salesItem):
    prodId = salesItem[0]
    prodInf = salesItem[1]
    return (prodId, prodInf['custCount'], prodInf['totalRev'])


output = list(map(convertInput, reduce(insertProduct, sales, {}).items()))
print(output)

[('P08719', 16, 1275.9900000000007), ('P56063', 17, 1142.9300000000007), ('P41420', 19, 1298.7900000000009), ('P58928', 17, 971.1100000000007), ('P04501', 18, 1139.0300000000007), ('P55500', 17, 949.1100000000006), ('P86411', 17, 870.1200000000006), ('P14419', 16, 990.0200000000007), ('P20036', 16, 1250.9200000000008), ('P99613', 15, 1128.9800000000007)]
